In [11]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.preprocessing import LabelEncoder, StandardScaler, LabelBinarizer
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import classification_report

In [12]:
df = pd.read_csv('default_credit.csv').drop(columns='index')
df.head()

,default,student,balance,income
0,No,No,729.526495,44361.625074
1,No,Yes,817.180407,12106.134700
2,No,No,1073.549164,31767.138947
3,No,No,529.250605,35704.493935
4,No,No,785.655883,38463.495879


In [13]:
df['default'] = LabelEncoder().fit_transform(df['default'])
df['student'] = LabelEncoder().fit_transform(df['student'])
df.head()

,default,student,balance,income
0,0,0,729.526495,44361.625074
1,0,1,817.180407,12106.134700
2,0,0,1073.549164,31767.138947
3,0,0,529.250605,35704.493935
4,0,0,785.655883,38463.495879


In [14]:
df['student'].value_counts()

0    7056
1    2944
Name: student, dtype: int64

Modelo Base

In [16]:
X_train, X_test, y_train, y_test = train_test_split(df.drop('default',axis=1),df['default'],test_size=.33,random_state=1789)

model_lda=LinearDiscriminantAnalysis().fit(X_train,y_train)
predicted_y= model_lda.predict(X_test)
print(classification_report(y_test, predicted_y))


              precision    recall  f1-score   support

           0       0.98      1.00      0.99      3198
           1       0.87      0.25      0.39       102

    accuracy                           0.98      3300
   macro avg       0.92      0.63      0.69      3300
weighted avg       0.97      0.98      0.97      3300



Para la clase '0' se observa que los verdaderos positivos se encontraron en su mayoria. No obstante para la clase '1' tambien el nivel de precision fue alto. 

En el caso de Recall, el total de verdaderos positivos se detectaron en un 100% para la clase '0'. Para la clase '1' la mayoria resultaron ser considerados falsos negativos. Por lo que el modleo no tuvo un buen desempeno en esta clase.

De acuerdo al f1-score se observa que para la clase '1' el modelo LDA no tiene buena predicion.

Refactorizacion con informacion a priori

In [21]:
#Cambio del hiperparametro priori

model_lda_50_50= LinearDiscriminantAnalysis(priors=[0.5,0.5]).fit(X_train,y_train)
predicted_y= model_lda_50_50.predict(X_test)
print(classification_report(y_test, predicted_y))

              precision    recall  f1-score   support

           0       1.00      0.85      0.92      3198
           1       0.16      0.88      0.27       102

    accuracy                           0.85      3300
   macro avg       0.58      0.87      0.59      3300
weighted avg       0.97      0.85      0.90      3300



In [31]:
model_lda_60_40= LinearDiscriminantAnalysis(priors=[0.6,0.4]).fit(X_train,y_train)
predicted_y= model_lda_60_40.predict(X_test)
print(classification_report(y_test, predicted_y))

              precision    recall  f1-score   support

           0       0.99      0.89      0.94      3198
           1       0.20      0.84      0.32       102

    accuracy                           0.89      3300
   macro avg       0.59      0.87      0.63      3300
weighted avg       0.97      0.89      0.92      3300



Modelo 50-50: Para la clase '0'  los valores de precision, recall son similares. Teniendo muy buen desempeno. La deteccion de true positive frente a false positive mejora frente al modelo original(precision).
El recall para la clase '0' baja, lo que indica que al cambiar las probabilidades no se logran identificar todos los true positives.

Para la clase '1' el recall mejora considerablemente, pero la precision baja mucho. Esto indica que este nuevo modelo falla en encontrar demasiados '1' que corresponden ser '0'. Pero encuentra la mayoria de los '1' que si son.



Modelo 60-40: Este modelo se parece al modleo 50-50, dado que para '0' baja el recall y la precision es  casi total. Para la clase '1' la precision sigue siendo muy baja y el recall sube bastante.

Refactorizacion Oversampling

In [29]:
from imblearn.over_sampling import SMOTE
# Instanciamos la clase
oversampler = SMOTE(random_state=11238)
# generamos el eversampling de la matriz de entrenamiento y
X_train_oversamp, y_train_oversamp = oversampler.fit_resample(X_train,
y_train)

model_lda_oversampling= LinearDiscriminantAnalysis(priors=[0.6,0.4]).fit(X_train_oversamp,y_train_oversamp)
predicted_y= model_lda_oversampling.predict(X_test)
print(classification_report(y_test, predicted_y))



              precision    recall  f1-score   support

           0       1.00      0.88      0.93      3198
           1       0.19      0.87      0.31       102

    accuracy                           0.88      3300
   macro avg       0.59      0.88      0.62      3300
weighted avg       0.97      0.88      0.91      3300



Al efectuar oversampling, su desempeno se asemeja al incluir informacion a priori. Teniendo desempenos similares a los modelos previos.



Refactorizacion 3 QDA(Quadratic Discrimination Model)

In [30]:
model_qda=QuadraticDiscriminantAnalysis().fit(X_train,y_train)
predicted_y= model_qda.predict(X_test)
print(classification_report(y_test, predicted_y))

              precision    recall  f1-score   support

           0       0.98      1.00      0.99      3198
           1       0.82      0.30      0.44       102

    accuracy                           0.98      3300
   macro avg       0.90      0.65      0.72      3300
weighted avg       0.97      0.98      0.97      3300



En cada uno de los modelos vistos, siempre existe alguna metrica con valores bastante bajos que no indican un m=bue desempeno del modelo en esa metrica.
En general el analisis discriminante lineal y cuadrarico logra predecir bien valores futuros para la clase 0, pero la clase 1, tiene bajo el recall o la precision. Considerando el promedio que se obtiene por el f1-score, el cual es mas alto para la clase 1 en relacion a los otros modelos, se podria decir que el QDA es el mejor.